In [68]:
import pandas as pd
import numpy as np
#import argparse
from itertools import groupby
from operator import itemgetter


In [97]:
#testing purposes
data = pd.read_csv('Teacher_103_14_03_04_Purple.csv-tracked-region-[335, 486, 716, 715].csv')

In [100]:
# Just have to look at "Teacher_103_14_03_04_Purple.csv-tracked-region-[335, 486, 716, 715].csv" during instances of joint exploration
# Look at instances of joint exploration based on the inferences that nessrine provides. (3 things: 20 second marks, teacher interacting with students, teaching interacting with group (and not group))
#   joint exploration time stamps: T103 14-03-04 (folder link).
#   This lesson has the Natural Log Joint Exploration Episode (~36:11 –40:13, Teacher interacting with Orange Group

# For file: # Just have to look at instances of joint exploration for "Teacher_103_14_04_22_Green.csv-tracked-region-[950, 475, 1893, 1072].csv"
# T103 14-04-22
# Cosine Joint Exploration Episode (~14:47-26:07, Teacher interacting with Orange Group)

# Pick and easily identifiable group

#data = pd.read_csv(input("Full path to csv file: "))

def main():
    df = data
    mean_people_changed_df = mean_people_changed(df)
    mean_headkp_y_val_df = mean_headkp_y_val(df)
    #variance_movement_per_person_df = variance_movement_per_person(df)
    eventframes = determine_event_frames(mean_people_changed_df,mean_headkp_y_val_df)
    return(eventframes)
    
def mean_people_changed(dataframe):
    """
    Returns the df of video frames (columns: mean_num_people  difference_in_people) 
    when the change in rolling mean of detected person counts across # frames is 
    ># people when calculating the difference between # frames
    """
    pplCntPerFrame = dataframe.groupby('frame_num',as_index=False).person_id.count()
    pplCntPerFrame['mean_num_ppl'] = pplCntPerFrame.person_id.rolling(window=90).mean()
    pplCntPerFrame['difference_in_people'] = abs(pplCntPerFrame.mean_num_ppl.diff(periods=90))
    candidate_frames = pplCntPerFrame[pplCntPerFrame.difference_in_people>1]
    print('\n mean_people_changed: done\n')
    print("\nframes where mean people changed is greater than 1: ",candidate_frames)
    return candidate_frames

def mean_headkp_y_val(dataframe):
    """
    Returns the df of video frames when the change in rolling mean of 
    y-values across y-head frames detected person counts across # frames is 
    in the 98th percentile of people when calculating the difference between # frames

    We should be looking at when the y-val is in the positive
    """
    #y_val_df = data[['frame_num', 'person_id','new_id','keypoint1_y', 'keypoint16_y',\
    #    'keypoint17_y', 'keypoint18_y', 'keypoint19_y']].copy()    # !!! this version assumes using new_id
    y_val_df = data[['frame_num', 'person_id','keypoint1_y', 'keypoint16_y',\
        'keypoint17_y', 'keypoint18_y', 'keypoint19_y']].copy()    
    y_val_df = y_val_df.replace(0,np.nan)
    y_val_df['headkp_mean'] = y_val_df[['keypoint1_y', 'keypoint16_y', 'keypoint17_y',\
        'keypoint18_y', 'keypoint19_y']].mean(axis=1)
    yvals_df = y_val_df.groupby('frame_num',as_index=False)['headkp_mean'].mean()
    yvals_df['rolling_mean_yvals'] = yvals_df['headkp_mean'].rolling(window=90).mean()
    yvals_df['difference_in_yval'] =  abs(yvals_df['rolling_mean_yvals'].diff(periods=90)) #keeping these the same as the rolling
    top10percentile = yvals_df['difference_in_yval'].quantile(.90)
    candidate_frames = yvals_df[yvals_df.difference_in_yval>=top10percentile]
    print('\n mean_headkp_y_val: done\n')
    print("\nlength of list of frames when y val is in the top 10 percentile: ",len(candidate_frames))
    print("\nframes where headkp y val is in the top 10 percentile: ",candidate_frames)
    return candidate_frames


#!!!!!!! NOTES ON WHY THIS FEATURE IS NOT CORRECTLY CALCULATING !!!!!!
#confusing because we are actually not calculating movement per frame
#Roll over overlapping windows
""" 
we want to iterate by frame rather than by rows (as in iterrows()) over the entire dataset. We want to groupby by frames rather than rows. 
    -create dataframes by subsetting by 90 frames worth of rows
        -relevant_cols_df[(relevant_cols_df.frame_num >= row.frame_num) & (relevant_cols_df.frame_num < row.frame_num + 90)]
    -then calculate std of each dataframe, then 
!!!!!!! NOTES ON WHY THIS FEATURE IS NOT CORRECTLY CALCULATING !!!!!!


def variance_movement_per_person(dataframe):
    relevant_cols_df = data.filter(regex=("frame_num|person_id|keypoint.{0,2}_x|keypoint.{0,2}_y"))
    movements = []
    for window in relevant_cols_df.rolling(90):
        movements.append(window.groupby('person_id').std().mean().mean())
    relevant_cols_df['movement_vars'] = movements
    relevant_cols_df['difference_in_movement_vars'] =  abs(relevant_cols_df.movement_vars.diff(periods=90))
    top10percentile = relevant_cols_df.difference_in_movement_vars.quantile(.90)
    candidate_frames = relevant_cols_df[relevant_cols_df.difference_in_movement_vars>=top10percentile]
    print('\nvariance_movement_per_person: done\n')
    print("\nlength of list of frames when y val is in the top 10 percentile: ",len(candidate_frames))
    print("\nframes where headkp y val is in the top 10 percentile: ",candidate_frames)
    return candidate_frames
    #closest_dist_cols['total_mvmnt_mag'] = closest_dist_cols.loc[:,[c for c in closest_dist_cols.columns if c!= "frame_num"]].sum(axis=1) #only consider instances when it's a new person ID. Do not use arbitrary approach using cutoff as 15*15 = 225
"""

def determine_event_frames(ppl_df,head_df):
    """
    Returns a list of suspected frames (consecutiveframes) from common frames found from features. 
    If a frames block is less than X consecutive frames, it is removed. Remove the block if another 
    frames block starts within X frames of the last block.
    Relevant info: HA's behavior sensitivity interactions were being coded at >1 min, interaction 
    with group was coded at 10 seconds.
    """
    commonframes = set(ppl_df.frame_num) & set(head_df.frame_num)
    #commonframes = list(commonframes)
    #commonframes.sort()
    consecutiveframes = []
    print("length of common frames: ",len(commonframes))
    print("common frames: ",commonframes)

    # code block below tested working assuming a "commonframes" list type object is passed. 
    # code above is currently assigned and printed as a set type object, but has weirdness changing it a list type object. 
    """
    # from: https://stackoverflow.com/questions/7731213/print-doesnt-print-when-its-in-map-python
    for k, g in groupby(enumerate(commonframes), lambda ix: ix[0] - ix[1]): 
        consecutiveframes.append(list(map(itemgetter(1), g)))

    for list in consecutiveframes:
        if len(list)<60: #if event block is less than 60 frames or 1 seconds
            consecutiveframes.remove(list)
        if (list in consecutiveframes) and (list!=consecutiveframes[-1]):
            if consecutiveframes[consecutiveframes.index(list)+1][0] - list[-1] < 300: #if next event starts within 10 seconds of current one
                consecutiveframes.remove(consecutiveframes[consecutiveframes.index(list)+1])
    return(consecutiveframes)
    """""
print(main())


 mean_people_changed: done


frames where mean people changed is greater than 1:         frame_num  person_id  mean_num_ppl  difference_in_people
206       6984.0          1      1.033333              1.022222
207       6985.0          1      1.022222              1.044444
208       6986.0          1      1.011111              1.066667
209       6987.0          1      1.000000              1.088889
210       6988.0          1      1.000000              1.088889
...          ...        ...           ...                   ...
24629    82138.0          3      2.144444              1.033333
24630    82139.0          2      2.144444              1.022222
24631    82140.0          2      2.144444              1.011111
24639    82148.0          2      2.188889              1.011111
24640    82149.0          2      2.200000              1.022222

[494 rows x 4 columns]

 mean_headkp_y_val: done


length of list of frames when y val is in the top 10 percentile:  2211

frames where headkp y val